In [1]:
%config IPCompleter.greedy=True

In [32]:
#!/usr/bin/env python
# coding: utf-8
import re

import spacy
import pandas as pd
from nltk.corpus import stopwords
from vaderSentimentimport SentimentIntensityAnalyzer
from gensim.summarization.textcleaner import split_sentences

from flashtext import KeywordProcessor

import pprint
# TextBlob Sentiment Analysis
from textblob import TextBlob
from nltk import word_tokenize
from rake_nltk import Rake,Metric

import json

SyntaxError: invalid syntax (<ipython-input-32-26384700f0a9>, line 8)

In [33]:
def load_data_from_json(json_file='generated_data.json', company_list=[]):
    with open(json_file, 'r') as f:
        json_data = json.load(f)
        
    data_set = []
    for company_data in json_data["information"]["companies"]:
        company_name = company_data["name"]
        if company_list ==[] or company_name in company_list:
            
            print("Company name : "+company_name)
            for yearly_data in company_data["years"]:
                print("year: "+str(yearly_data["year"]))
                news = yearly_data['news']
                cleaned_news = '. '.join(news.split('\n'))
                quarters = yearly_data["quarters"]
                
                for quarter in quarters:
                    speech = ""
                    print("Quarter: " + str(quarter["quarter"]))
                    transcript = quarter["transcript"]
                    print("Title: " + transcript["title"])
                    
                    for each in transcript['text']:
                        speech += ' ' + ' '.join(each['speech']) 
                    text_data = speech # + ' ' + cleaned_news
                    data_set.append({"text": text_data,
                                 "title": transcript["title"],
                                 "quarter": quarter["quarter"],
                                 "year": yearly_data["year"],
                                 "company": company_name,
                                 "gross_profit": quarter["gross_profit"],
                                 "net_income_loss": quarter["net_income_loss"],
                                 "eps_basic": quarter["eps_basic"]}
                                )
    print("Done.")
    return data_set

In [34]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [65]:
keyword_dict ={
    "technology":["tech"],
    "accelerators":["accelerators"],
    "4g":["4g","4 g"], 
    "5g":["5g","5 g"],
    "wireless":["wifi","wi-fi","wireless"],
    "smart cell":["smart cell"],
    "gig":["800gig","800-gig","600gig","600g","10g","100g"],
    "nvme":["nvme"],
    "sonet":["sonet","sdh"],
    "integrated circuits":["plc","plcs","circuit","circuits"],
    "ai":["m/l","Machine Learning","AI/ML"],
    "architecture":["architecture"],
    "processor":["arm","processor"],
    "asics":["ASIC","chip","microchip","fpga"],
    "back haul": ["haul","backhaul","black-haul"],
    "baseband":["baseband"], 
    "compute":["compute"],
    "connected auto" :["connected auto","auto"],
    "connectivity":["connectivity"],
    "controller":["controller","controllers","control plane"],
    "data centre":["data centre","datacentre"],
    "datacom":["datacom"], 
    "dfe":["dfe"], 
    "signal processing":["dsp","digital signal processing"], 
    "ethernet":["ethernet"],
    "fabric":["fabric","fab"], 
    "firewall":["security","fire wall"], 
    "form factor":["form factor"], 
    "fpga":["fpga","firmware"],
    "front haul":["fronthaul"], 
    "fusion":["fusion"], 
    "gateways":["gateway"],
    "ghz":["ghz"], 
    "hdd":["hdd","disk"], 
    "bandwidth":["high bandwidth"], 
    "semiconductor": ["high speed","hyper scale"],
    "demand":["demand","high expectation","expectation","anticipation"]
    "gaap":["gaap","nongaap","non-gaap"],
    "guidence":["direction"],
    "growth":["growth", "growing","strong","good","stength"],
    "gains":["gain","gained"],
    "margin drop":["margin","low margin","negetive margin","decline"],
    "opex":["opex","operations expenditure"],
    "capex":["capex","capital expenditure","expenditure"],
    "shipment":["shipment","export","ship"],
    "weak":["weak","weakness"],
    "fiscal quarter":["fiscal quarter","qtr","quarter"],
    "profit":["profit","gross profit", "profitable", "profits", "profitability", "profitably", "unprofitable"],
    "savings":["savings"],
    "layoff":["lay-off","attrition"],
    "optical":["optical","optic fiber"],
    "efficiency":["efficiency","optimization","increase efficiency","efficient"],
    "revenue":["revenue","income"],
    "eps":["eps","earning per share"],
    "Pandemic":["covid","corona","virus","covid-19"],
    "cloud":["cloud","cloud model","cloud business","cloud infra"],
    "acquisition":["acquire","acquired","acquisition","merger"],
    "divested":["divest","divestment","divested"],
    "opportunity":["opportunity","opportunities"],
    "upward":["increase","upward"],
}   

SyntaxError: invalid syntax (<ipython-input-65-4a76d1fb7a2c>, line 39)

In [66]:
keyword_processor = KeywordProcessor(case_sensitive=False)
keyword_processor.add_keywords_from_dict(keyword_dict)

In [67]:
def text_cleaner(text,num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [68]:
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english')) 
def raw_tokenize(text):
    paragraphs = text.split('\n')
    processed_text = ""
    for p in paragraphs:
        doc = nlp(p)
        for token in doc:
            processed_text += (token.text.lower() + " ")

    processed_text = re.sub(r"(thank you)", "", processed_text)
    processed_text = re.sub(r"(thank(s)*)", "", processed_text)
    processed_text = re.sub(r"(great*.)", "", processed_text)
    processed_text = re.sub(r"(superb.)", "", processed_text)
    processed_text = re.sub(r"(perfect.)", "", processed_text)
    processed_text = re.sub(r"(yes.)", "", processed_text)
    processed_text = re.sub(r"(sure.)", "", processed_text)
    processed_text = re.sub(r"(sure,yes.)", "", processed_text)
    processed_text = re.sub(r"(okay.)", "", processed_text)
    processed_text = re.sub(r"(good.)", "", processed_text)
    processed_text = re.sub(r"(good afternoon)", "", processed_text)
    processed_text = re.sub(r"(appreciate it.)", "", processed_text)
    processed_text = re.sub(r"(happy answer.)", "", processed_text)
    
    
    return processed_text

In [69]:

def tokenize(text):
    paragraphs = text.split('\n')
    processed_text = ""
    for p in paragraphs:
#         if len(p) < 15:
#             continue
        doc = nlp(p)
        for token in doc:
            if token.text not in stop_words:
                if not token.is_punct:
                    replaced_tok = token.lemma_
                    if token.lemma_ == '-PRON-':
                        replaced_tok = 'it'
                    processed_text += (replaced_tok.lower() + " ")
                else:
                    processed_text += (token.text.lower() + " ")

    processed_text = re.sub(r"(thank you)", "", processed_text)
    processed_text = re.sub(r"(thank(s)*)", "", processed_text)
    processed_text = re.sub(r"(great.)", "", processed_text)
    processed_text = re.sub(r"(superb.)", "", processed_text)
    processed_text = re.sub(r"(good afternoon)", "", processed_text)
    
    return processed_text

In [70]:
def summary_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

In [71]:
def filter_financial_df(df_all, search_column= "Cleaned"):
    #Filter all Financial Key Words 
    df_finance =df_all[df_all[search_column].str.contains("margin")| 
                       df_all[search_column].str.contains("revenue")|
                       df_all[search_column].str.contains("margin")|
                       df_all[search_column].str.contains("gaap")|
                       df_all[search_column].str.contains("eps")|
                       df_all[search_column].str.contains("compensation")|
                       df_all[search_column].str.contains("acquisition")|
                       df_all[search_column].str.contains("capex")|
                       df_all[search_column].str.contains("assets")|
                       df_all[search_column].str.contains("Liability")|
                       df_all[search_column].str.contains("growth")|
                       df_all[search_column].str.contains("warning")|
                       df_all[search_column].str.contains("guidence")|
                       df_all[search_column].str.contains("tangible")|
                       df_all[search_column].str.contains("ebitda")|
                       df_all[search_column].str.contains("performance")|
                       df_all[search_column].str.contains("financial strength")|
                       df_all[search_column].str.contains("demand")
                      ]
    return df_finance

def filter_technology_df(df_all, search_column= "Cleaned"):
    #Filter technical keywords.
    
    df_tech =df_all[df_all[search_column].str.contains("5g")| 
                        df_all[search_column].str.contains("accelerators ")|
                        df_all[search_column].str.contains("ai")|
                        df_all[search_column].str.contains("asic")|
                        df_all[search_column].str.contains("back haul")|
                        df_all[search_column].str.contains("backhaul")|
                        df_all[search_column].str.contains("dsp")|
                        df_all[search_column].str.contains("controller")|
                        df_all[search_column].str.contains("dfe")|
                        df_all[search_column].str.contains("ethernet")|
                        df_all[search_column].str.contains("fabric")|
                        df_all[search_column].str.contains("firewall")|
                        df_all[search_column].str.contains("fpga")|
                        df_all[search_column].str.contains("front haul")|
                        df_all[search_column].str.contains("fushion")|
                        df_all[search_column].str.contains("gateway")|
                        df_all[search_column].str.contains("ghz")|
                        df_all[search_column].str.contains("hdd")|
                        df_all[search_column].str.contains("hyper scale")|
                        df_all[search_column].str.contains("infrastructure")|
                        df_all[search_column].str.contains("mimo")|
                        df_all[search_column].str.contains("nanometer")|
                        df_all[search_column].str.contains("multicore")|
                        df_all[search_column].str.contains("nvme")|
                        df_all[search_column].str.contains("optical")|
                        df_all[search_column].str.contains("packet processing")|
                        df_all[search_column].str.contains("pcle")|
                        df_all[search_column].str.contains("phy")|
                        df_all[search_column].str.contains("power")|
                        df_all[search_column].str.contains("preamps")|
                        df_all[search_column].str.contains("processor")|
                        df_all[search_column].str.contains("communication")|
                        df_all[search_column].str.contains("laser")|
                        df_all[search_column].str.contains("3d sensing")|
                        df_all[search_column].str.contains("cloud")|
                        df_all[search_column].str.contains("datacom")|
                        df_all[search_column].str.contains("telecom")|
                        df_all[search_column].str.contains("gig")|
                        df_all[search_column].str.contains("800 gig")|
                        df_all[search_column].str.contains("50g")|
                        df_all[search_column].str.contains("dml")|
                        df_all[search_column].str.contains("PAM")|
                        df_all[search_column].str.contains("lithium")|
                        df_all[search_column].str.contains("gallium")|
                        df_all[search_column].str.contains("indium")|
                    df_all[search_column].str.contains("sonet")|
                    df_all[search_column].str.contains("data centre")|
                    df_all[search_column].str.contains("design in")|
                    df_all[search_column].str.contains("data win")|
                    df_all[search_column].str.contains("front haul")
                      ]
    return df_tech


In [72]:
def find_word_a_country(text):
    words = word_tokenize(text)  
    countries =[]
    for word in words:
        try:
            country= pycountry.countries.lookup(word)
            countries.append(country.name)
        except:
            country=""
    return countries

In [73]:
def getKeyPhrases(sentence):
    r_ake= Rake(ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,min_length=3)
    r_ake.extract_keywords_from_text(sentence)
    keyPhrases = r_ake.get_ranked_phrases()
    keyPhrases=','.join([str(elem).title() for elem in keyPhrases])
    return keyPhrases

In [74]:
def assign_sentiment_data(sentence,cleanedSentene,cnt,vs):
    
    tb= TextBlob(sentence.lower())
    vs['polarity'] = tb.sentiment.polarity
    vs['subjectivity'] = tb.sentiment.subjectivity
#     vs['intensity'] = tb.sentiment.intensity
    k_words =keyword_processor.extract_keywords(sentence.lower())
    countries_in_sent = find_word_a_country(sentence.lower())
    k_words.extend(countries_in_sent)

    #Assign Marked Key words 
    vs['keywords']= ' '.join([str(elem).title() for elem in k_words])
    vs['KeyPhrases']= getKeyPhrases(sentence)

In [75]:
def generate_vad_sentiment(df_sentence, quarter, company,year, category):
    #cleaned_sentences = [tokenize(sentence) for sentence in df_sentence["Source"]]
    sentences = df_sentence["Source"]
    analyzer = SentimentIntensityAnalyzer()
    sentence_list=[]

    cnt=0
    for sentence in sentences:
        cleanedSentene = text_cleaner(sentence,0)
        vs = analyzer.polarity_scores(raw_tokenize(sentence))
        vs["pos"] = vs["pos"]/vs["neu"]
        vs["neg"] = vs["neg"]/vs["neu"]
        vs['Cleaned']=cleanedSentene
        vs["Company"]=company
        vs["Quarter"]=quarter
        vs["Category"]=category
        vs["Year"]=year
        vs['Source_row_id']=df_sentence.index.values[cnt] #This one was tricky ,no easy documentation
        assign_sentiment_data(sentence,cleanedSentene,cnt,vs)
        sentence_list.append(vs)
        cnt+=1
        
    return sentence_list


In [76]:
def construct_scores_frame(df_sentence,sentence_list):        
    df_score = pd.DataFrame(sentence_list)
    df_score.reset_index(drop=True, inplace=True)
    sen_list = df_sentence['Source']
    sentence_series = pd.Series(sen_list)
    
    df_score['Source']=sentence_series.values
    df_score= df_score[['Company','Quarter','Category','Year','KeyPhrases','Source','Cleaned','pos','neg','neu','compound','Source_row_id','polarity','subjectivity','keywords']]
    df_score.sort_values(by=['pos'], inplace=True, ascending=False)
    return df_score

In [77]:
def process_article(raw_data):
    i=0
    for raw_article in raw_data:
        print("Process Article "+str(i+1)+":"+raw_article["title"])
        
        print("stage 1: splite row article into list of sentence")
        raw_sentences = list(set(split_sentences(raw_article["text"])))
        
        print("stgae 2: create data frame")

        df_all= pd.DataFrame(raw_sentences,columns=['Source'])

        print("stage 3: create data frame for financial and tech ")
        fin_df = filter_financial_df(df_all, search_column="Source")
        tech_df = filter_technology_df(df_all,  search_column="Source")

        quarter = raw_article["quarter"]     
        company = raw_article["company"]     
        year = raw_article["year"]     

        print("stage 5: Apply sentiment analyser on top of financial and tech df")
        fin_sentiment=generate_vad_sentiment(fin_df, quarter, company, year, category="Financial")
        tech_sentiment=generate_vad_sentiment(tech_df, quarter, company, year, category="Technology")

        
        print("stage 6: convert the sentiment data to dataframe")
      
        fin_df_sentiment=construct_scores_frame(fin_df, fin_sentiment)
        tech_df_sentiment=construct_scores_frame(tech_df, tech_sentiment)


        print("stage 7: stored in to a final dataframe ")
        print("")
        if i==0:
            final_df=fin_df_sentiment
            final_df = final_df.append(tech_df_sentiment)
        else:
            final_df = final_df.append(fin_df_sentiment)
            final_df = final_df.append(tech_df_sentiment)
        i = i + 1
    print("Finished")
    return final_df

In [79]:
data = load_data_from_json('generated_data.json', company_list=["MRVL"])

Company name : MRVL
year: 2017
Quarter: -1
Title: MRVL - Conference call 2017
Quarter: 3
Title: MRVL - Earnings call Q3 2017
Quarter: 2
Title: MRVL - Earnings call Q2 2017
year: 2018
Quarter: -1
Title: MRVL - Conference call 2018
Quarter: 4
Title: MRVL - Earnings call Q4 2018
Quarter: 3
Title: MRVL - Earnings call Q3 2018
Quarter: 2
Title: MRVL - Earnings call Q2 2018
Quarter: 1
Title: MRVL - Earnings call Q1 2018
year: 2019
Quarter: -1
Title: MRVL - Conference call 2019
Quarter: 4
Title: MRVL - Earnings call Q4 2019
Quarter: 3
Title: MRVL - Earnings call Q3 2019
Quarter: 2
Title: MRVL - Earnings call Q2 2019
Quarter: 1
Title: MRVL - Earnings call Q1 2019
year: 2020
Quarter: 4
Title: MRVL - Earnings call Q4 2020
Quarter: 3
Title: MRVL - Earnings call Q3 2020
Quarter: 2
Title: MRVL - Earnings call Q2 2020
Quarter: 1
Title: MRVL - Earnings call Q1 2020
Done.


In [80]:
df = process_article(data)

Process Article 1:MRVL - Conference call 2017
stage 1: splite row article into list of sentence
stgae 2: create data frame
stage 3: create data frame for financial and tech 
stage 5: Apply sentiment analyser on top of financial and tech df
stage 6: convert the sentiment data to dataframe
stage 7: stored in to a final dataframe 

Process Article 2:MRVL - Earnings call Q3 2017
stage 1: splite row article into list of sentence
stgae 2: create data frame
stage 3: create data frame for financial and tech 
stage 5: Apply sentiment analyser on top of financial and tech df
stage 6: convert the sentiment data to dataframe
stage 7: stored in to a final dataframe 

Process Article 3:MRVL - Earnings call Q2 2017
stage 1: splite row article into list of sentence
stgae 2: create data frame
stage 3: create data frame for financial and tech 
stage 5: Apply sentiment analyser on top of financial and tech df
stage 6: convert the sentiment data to dataframe
stage 7: stored in to a final dataframe 

Proce

In [81]:
fin_sentiment = df[(df.Category == "Financial")]
fin_sentiment.head(50)

,Company,Quarter,Category,Year,KeyPhrases,Source,Cleaned,pos,neg,neu,compound,Source_row_id,polarity,subjectivity,keywords
17,MRVL,-1,Financial,2017,Comprehensive Solutions Portfolio Delivers Per...,This comprehensive solutions portfolio deliver...,comprehensive solutions portfolio delivers per...,0.524390,0.000000,0.656,0.6486,134,0.000000,0.000000,
41,MRVL,-1,Financial,2017,"Term Top Line Growth Challenges,Fiber Channel ...",Both HDD and fiber channel franchises are numb...,hdd fiber channel franchises number one market...,0.424501,0.000000,0.702,0.8018,388,0.575000,0.575000,Hdd Growth
21,MRVL,-1,Financial,2017,Improved Execution Across,Our strong performance in Q3 is a direct resul...,strong performance direct result continued gro...,0.408451,0.000000,0.710,0.8402,170,0.177778,0.586111,Growth Growth
3,MRVL,-1,Financial,2017,High Growth Data Center Markets,"In addition, the combined company will have th...",addition combined company comprehensive soluti...,0.404494,0.000000,0.712,0.7070,28,0.186667,0.380000,Growth
23,MRVL,-1,Financial,2017,Diversified End Market Opportunities Leveraged...,"In addition, we will have a best-in-class fina...",addition best class financial model compelling...,0.364568,0.099561,0.683,0.8020,197,0.077273,0.300000,Revenue Growth Margin Drop Opportunity
47,MRVL,-1,Financial,2017,"Help Drive Strong Growth,Entire Infrastructure...",We have developed an incredible portfolio of p...,developed incredible portfolio products ip cus...,0.358696,0.000000,0.736,0.8225,407,0.126667,0.711667,Growth Growth
20,MRVL,-1,Financial,2017,"Single Digit Grower Based,Proven Track Record","But when we look at it and we roll it up, we f...",look roll feel comfortable high single digit g...,0.353180,0.000000,0.739,0.9292,157,0.520000,1.000000,Growth
28,MRVL,-1,Financial,2017,"Two Switching Portfolios,Strong Position Today...",I think the -- we are pretty excited about the...,think pretty excited combination two switching...,0.280410,0.000000,0.781,0.9273,233,0.396111,0.901111,Growth Growth Opportunity
44,MRVL,-1,Financial,2017,,So we think when we map it out we look at wher...,think map look strong engaged storage example ...,0.248439,0.000000,0.801,0.9169,399,0.247222,0.613889,Growth Growth Ethernet Processor Revenue
33,MRVL,-1,Financial,2017,"Combined Company Revenue Currently,Clearly See...",With combined company revenue currently at app...,combined company revenue currently approximate...,0.248439,0.000000,0.801,0.7713,311,-0.020000,0.481667,Revenue Growth


In [82]:
fin_sentiment[['Quarter', 'KeyPhrases', 'Source', 'pos', 'polarity', 'keywords']].query('pos > 0.2')

,Quarter,KeyPhrases,Source,pos,polarity,keywords
17,-1,Comprehensive Solutions Portfolio Delivers Per...,This comprehensive solutions portfolio deliver...,0.524390,0.000000,
41,-1,"Term Top Line Growth Challenges,Fiber Channel ...",Both HDD and fiber channel franchises are numb...,0.424501,0.575000,Hdd Growth
21,-1,Improved Execution Across,Our strong performance in Q3 is a direct resul...,0.408451,0.177778,Growth Growth
3,-1,High Growth Data Center Markets,"In addition, the combined company will have th...",0.404494,0.186667,Growth
23,-1,Diversified End Market Opportunities Leveraged...,"In addition, we will have a best-in-class fina...",0.364568,0.077273,Revenue Growth Margin Drop Opportunity
...,...,...,...,...,...,...
54,1,"Tremendous Engineering Talent,Also Equally Exc...",I am also equally excited about the tremendous...,0.303781,0.354167,
49,1,,In November of 2017 we announced our intent to...,0.250000,0.433333,Growth Acquisition
66,1,New Custom Products Including Full Asic,This acquisition was motivated by the growing ...,0.246883,0.243182,Acquisition Growth Asics
7,1,"Single Digit Revenue Customers,One Like Mid",Huawei is only one like mid-single digit reven...,0.226994,0.000000,Revenue


In [83]:
df_tech = df[(df.Category == "Technology")]
df_tech

,Company,Quarter,Category,Year,KeyPhrases,Source,Cleaned,pos,neg,neu,compound,Source_row_id,polarity,subjectivity,keywords
19,MRVL,-1,Technology,2017,Comprehensive Solutions Portfolio Delivers Per...,This comprehensive solutions portfolio deliver...,comprehensive solutions portfolio delivers per...,0.524390,0.000000,0.656,0.6486,134,0.000000,0.000000,
24,MRVL,-1,Technology,2017,,We will share more details on our progress in ...,share details progress months ahead,0.453488,0.000000,0.688,0.6124,149,0.500000,0.500000,
17,MRVL,-1,Technology,2017,,But I would say we made progress in more targe...,would say made progress targeted fashion certa...,0.369863,0.000000,0.730,0.8625,125,0.238095,0.690476,Cloud
34,MRVL,-1,Technology,2017,Diversified End Market Opportunities Leveraged...,"In addition, we will have a best-in-class fina...",addition best class financial model compelling...,0.364568,0.099561,0.683,0.8020,197,0.077273,0.300000,Revenue Growth Margin Drop Opportunity
78,MRVL,-1,Technology,2017,"Help Drive Strong Growth,Entire Infrastructure...",We have developed an incredible portfolio of p...,developed incredible portfolio products ip cus...,0.358696,0.000000,0.736,0.8225,407,0.126667,0.711667,Growth Growth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,MRVL,1,Technology,2020,,Is that kind of the right way to think about t...,kind right way think overall trajectory busine...,0.000000,0.000000,1.000,0.0000,139,0.295238,0.478571,
32,MRVL,1,Technology,2020,,Our cloud position there is small.,cloud position small,0.000000,0.000000,1.000,0.0000,192,-0.250000,0.400000,Cloud
57,MRVL,1,Technology,2020,,"And with that, I'll turn the call over to Jean...",turn call jean details recent results outlook,0.000000,0.000000,1.000,0.0000,331,0.250000,0.375000,
27,MRVL,1,Technology,2020,"Investor Relations Section,Gaap Financial Meas...",A reconciliation between our GAAP and non-GAAP...,reconciliation gaap non gaap financial measure...,0.000000,0.000000,1.000,0.0000,153,0.200000,0.200000,Gaap Gaap


In [84]:
fin_sentiment[['Year', 'Quarter', 'KeyPhrases', 'Source', 'pos', 'neg', 'subjectivity', 'polarity', 'keywords']].query("pos > 0.2 and subjectivity < 0.4 and Year >= 2020 and Quarter == 3 and keywords.str.contains('Profit')", engine='python').sort_values(by = 'pos', ascending = False).style.set_properties(**{'text-align':'left'})


,Year,Quarter,KeyPhrases,Source,pos,neg,subjectivity,polarity,keywords
45,2020,3,"Term Profitable Growth,Remaining Synergy Achievement,Operating Efficiency Improvement","We will continue to drive the remaining synergy achievement and the operating efficiency improvement through fiscal 2021, while investing in a growth initiative to position us to deliver long-term profitable growth.",0.453488,0.000000,0.000000,0.000000,Efficiency Growth Profit Growth
92,2020,3,Gaap Operating Profit,Non-GAAP operating profit was $138 million or 21% of revenue.,0.207729,0.000000,0.000000,0.000000,Gaap Profit Revenue


In [87]:
#fin_sentiment.nlargest(5,['pos','polarity'])

In [100]:
tech_sentiment = df[(df.Category == "Technology") &(df.Year >2020)]
tech_sentiment

,Company,Quarter,Category,Year,KeyPhrases,Source,Cleaned,pos,neg,neu,compound,Source_row_id,polarity,subjectivity,keywords


In [92]:
tech_sentiment[['Year', 'Quarter', 'KeyPhrases', 'Source', 'pos', 'neg', 'subjectivity', 'polarity', 
                'keywords']].query("pos > 0.2 and subjectivity < 0.4 and Year >= 2021 and Quarter == 1", engine='python').sort_values(by = 'pos', ascending = False).style.set_properties(**{'text-align':'left'})

,Year,Quarter,KeyPhrases,Source,pos,neg,subjectivity,polarity,keywords


In [93]:
tech_sentiment[['Year', 'Quarter', 'KeyPhrases', 'Source', 'pos', 'neg', 'subjectivity', 'polarity', 'keywords']].query("pos > 0.2 and subjectivity < 0.4 and Year >= 2020 and Quarter == 1", engine='python').sort_values(by = 'pos', ascending = False).style.set_properties(**{'text-align':'left'})

,Year,Quarter,KeyPhrases,Source,pos,neg,subjectivity,polarity,keywords
69,2020,1,"First Design Win,5G Remote Radiohead","Second and even more exciting, we now have our first design win for our processor in the 5G remote radiohead itself.",0.331558,0.000000,0.372222,0.291667,Processor 5G
56,2020,1,Said Something Like Fourth Quarter,"But, I think you said something like fourth quarter of this fiscal year.",0.231527,0.000000,0.000000,0.000000,Fiscal Quarter
34,2020,1,,In the meantime we remain focused on efficiently managing our storage business and continuing to pivot this business towards the enterprise and datacenter market.,0.230012,0.000000,0.000000,0.000000,
1,2020,1,"Second Tier 1 Oem Design Win,Last Earnings Call,Fusion Baseband Processor,Early Calendar 2020","As I mentioned in our last earnings call, we secured a second Tier 1 OEM design win for our Fusion baseband processor and I'm pleased to report that we are on schedule to sample the semi-custom product in early calendar 2020.",0.218027,0.000000,0.353333,0.280000,Fusion Baseband Processor


In [94]:
tech_sentiment[['Year', 'Quarter', 'KeyPhrases', 'Source', 'pos', 'neg', 'subjectivity', 'polarity', 'keywords']].query("pos > 0.2 and subjectivity < 0.4 and Year >= 2020 and Quarter == 1 and keywords.str.contains('Datacom')", engine='python').sort_values(by = 'pos', ascending = False).style.set_properties(**{'text-align':'left'})

,Year,Quarter,KeyPhrases,Source,pos,neg,subjectivity,polarity,keywords
